In [1]:
!pip install pyspark
!pip install pyarrow
!pip install -q findspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark=SparkSession.builder.appName("Linearreg").getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
data=spark.read.format("libsvm").load("/content/sample_linear_regression_data.txt")

In [6]:
data.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [7]:
lr=LinearRegression(featuresCol="features",labelCol="label",predictionCol="prediction")

In [8]:
lrmodel=lr.fit(data)

In [9]:
lrmodel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [10]:
lrmodel.intercept

0.14228558260358093

In [11]:
training_summary=lrmodel.summary

In [12]:
training_summary.r2

0.027839179518600154

In [13]:
training_summary.rootMeanSquaredError

10.16309157133015

In [14]:
train_data,test_data=data.randomSplit([0.7,0.3])

In [15]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                355|
|   mean|0.11746726344627402|
| stddev|  10.64757984978709|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [16]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                146|
|   mean| 0.5958932972346749|
| stddev|  9.494959550819729|
|    min|-22.949825936196074|
|    max| 22.923352376063196|
+-------+-------------------+



In [17]:
new_model=lr.fit(train_data)

In [18]:
test_res=new_model.evaluate(test_data)

In [19]:
test_res.rootMeanSquaredError

9.774994937888595

In [20]:
test_res.r2

-0.06716530665585885

In [21]:
unlabeled_data=test_data.select("features")

In [22]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [23]:
predictions=new_model.transform(unlabeled_data)

In [24]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  1.7861421733263048|
|(10,[0,1,2,3,4,5,...|  3.4430839336109047|
|(10,[0,1,2,3,4,5,...|-0.23758251888260215|
|(10,[0,1,2,3,4,5,...|     2.6141627617111|
|(10,[0,1,2,3,4,5,...| -0.9062573117817067|
|(10,[0,1,2,3,4,5,...| -0.8161426061328416|
|(10,[0,1,2,3,4,5,...|  1.8579688869715496|
|(10,[0,1,2,3,4,5,...|    3.75067935908516|
|(10,[0,1,2,3,4,5,...|  3.3656991503044758|
|(10,[0,1,2,3,4,5,...|  -2.149134014081599|
|(10,[0,1,2,3,4,5,...| -0.9539547303180897|
|(10,[0,1,2,3,4,5,...|  1.0456199410219944|
|(10,[0,1,2,3,4,5,...| -0.3972370153203306|
|(10,[0,1,2,3,4,5,...| -0.5765796375627079|
|(10,[0,1,2,3,4,5,...|  0.5179416698566721|
|(10,[0,1,2,3,4,5,...|  1.3805877844461025|
|(10,[0,1,2,3,4,5,...|   2.139318173472389|
|(10,[0,1,2,3,4,5,...| -1.3059884896048173|
|(10,[0,1,2,3,4,5,...|  -1.418947426468043|
|(10,[0,1,2,3,4,5,...|  -2.11147